In [1]:
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/images/concrete_crack_images_for_classification.zip -P /resources/data

--2020-10-03 16:53:15--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/images/concrete_crack_images_for_classification.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 245259777 (234M) [application/zip]
Saving to: ‘/resources/data/concrete_crack_images_for_classification.zip.11’

concrete_crack_imag 100%[===================>] 233.90M  26.1MB/s    in 12s     

2020-10-03 16:53:27 (20.1 MB/s) - ‘/resources/data/concrete_crack_images_for_classification.zip.11’ saved [245259777/245259777]



In [2]:
!unzip -n  /resources/data/concrete_crack_images_for_classification.zip -d  /resources/data

Archive:  /resources/data/concrete_crack_images_for_classification.zip


In [1]:
from PIL import Image
import matplotlib.pyplot as plt
import os
import glob
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.nn as nn
from torch import optim 

In [2]:
# DATASET

In [3]:
class Dataset(Dataset):
    
    #constructor
    def __init__(self, transform=None, train=True):
        directory = "/resources/data"
        positive = "Positive"
        negative = "Negative"
        
        positive_file_path = os.path.join(directory, positive)
        negative_file_path = os.path.join(directory, negative)
        
        positive_files = [os.path.join(positive_file_path, file) for file in os.listdir(positive_file_path) if file.endswith(".jpg")]
        positive_files.sort()
        negative_files = [os.path.join(negative_file_path, file) for file in os.listdir(negative_file_path) if file.endswith(".jpg")]
        negative_files.sort()
        
        number_of_samples = len(positive_files) + len(negative_files)
        self.all_files = [None] * number_of_samples
        self.all_files[::2] = positive_files
        self.all_files[1::2] = negative_files
        
        # Transform is going to be used on image
        self.transform = transform
        
        # torch.LongTensor
        self.Y = torch.zeros([number_of_samples]).type(torch.LongTensor)
        self.Y[::2] = 1
        self.Y[1::2] = 0
        
        if train:
            self.all_files = self.all_files[0:30000]
            self.Y = self.Y[0:30000]
            self.len = len(self.all_files)
        else:
            self.all_files = self.all_files[30000:]
            self.Y = self.Y[30000:]
            self.len = len(self.all_files)
    
    # get length
    def __len__(self):
        return self.len
    
    # getter
    def __getitem__(self, idx):
        image = Image.open(self.all_files[idx])
        y = self.Y[idx]
        
        # If there is any transform method, apply it onto the image
        if self.transform:
            image = self.transform(image)
            
        return image, y

In [4]:
# TRANSFORM OBJECT AND DATASET OBJECT

In [5]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean, std)])

In [6]:
dataset_train = Dataset(transform = transform, train = True)
dataset_val = Dataset(transform = transform, train = False)

In [7]:
dataset_train[0][0].shape
#dataset_train.view(-1, 154587)

torch.Size([3, 227, 227])

In [8]:
size_of_image = 3 * 227 * 227
size_of_image

154587

In [9]:
torch.manual_seed(0)

In [10]:
# CUSTOM MODULE

In [11]:
#class Net(nn.Module):
    
#    def __init__(self, in_size= size_of_image, out_size=2):
#        super(Net,self).__init__()
#        self.linear1 = nn.Linear(in_size, out_size)
    
#    def forward(self, x):
#        #x = self.linear1(x)
#        x = x.view(-1, 227 * 227*3)
#        x = self.linear1(x)
        
#        return x

In [12]:
class Model(nn.Module):
    
    def __init__(self, in_size = size_of_image, out_size = 2, bias = None):
        super(Model, self).__init__()
        self.linear1 = nn.Linear(in_size, out_size)
        
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = x.view(-1, 3 * 227 * 227)
        x = self.linear1(x)
        x = self.softmax(x)
        
        return x

In [13]:
# MODEL OBJECT

In [14]:
model = Model()
model.state_dict()

OrderedDict([('linear1.weight',
              tensor([[-1.9042e-05,  1.3644e-03, -2.0933e-03,  ..., -2.2041e-03,
                       -1.6752e-03, -6.4412e-04],
                      [ 5.9658e-04,  1.8556e-03, -9.0110e-04,  ..., -3.9324e-04,
                       -4.3716e-04, -3.5319e-04]])),
             ('linear1.bias', tensor([-0.0004, -0.0008]))])

In [15]:
# OPTIMIZER

In [16]:
optimizer = torch.optim.SGD(model.parameters(), lr = 0.1, momentum = 0.1)

In [17]:
#CRITERION

In [18]:
criterion = nn.CrossEntropyLoss()

In [19]:
# DATALOADER TRAINING AND VALIDATION

In [20]:
train_loader = DataLoader(dataset = dataset_train, batch_size = 1000)
val_loader = DataLoader(dataset = dataset_val, batch_size = 1000)

In [21]:
# TRAIN MODEL WITH 5 EPOCH

In [ ]:
epochs = 5
LOSS = []
ACCURACY = []
correct = 0
n_test = len(dataset_val)
#Softmax_fn=nn.Softmax(dim=-1)

def train_model(epochs):
    for epoch in range(epochs):
        loss_sub = []
        for x, y in train_loader:
            optimizer.zero_grad()
            z = model(x)
            loss = criterion(z, y)
            loss_sub.append(loss.data.item())
            loss.backward()
            optimizer.step()
        LOSS.append(np.mean(loss_sub))
        #print("probability of class(training) ", torch.max(Softmax_fn(z)).item())


        
        correct = 0
        for x_test, y_test in val_loader:
            z = model(x_test)
            _, yhat = torch.max(z.data, 1)
            correct += (yhat == y_test).sum().item()
        accuracy = correct / n_test
        ACCURACY.append(accuracy)
        print('Epoch ' + str(epoch) + ':' + str(accuracy))
        #print("probability of class(testing) ", torch.max(Softmax_fn(z)).item())


    
train_model(epochs)